In [ ]:
!pip install crewai crewai_tools langchain_community -qU

Defaulting to user installation because normal site-packages is not writeable


ERROR: Ignored the following yanked versions: 0.165.0
ERROR: Ignored the following versions that require a different python version: 0.100.0 Requires-Python >=3.10,<3.13; 0.100.1 Requires-Python >=3.10,<3.13; 0.102.0 Requires-Python >=3.10,<3.13; 0.105.0 Requires-Python >=3.10,<3.13; 0.108.0 Requires-Python >=3.10,<3.13; 0.114.0 Requires-Python >=3.10,<3.13; 0.117.0 Requires-Python >=3.10,<3.13; 0.117.1 Requires-Python >=3.10,<3.13; 0.118.0 Requires-Python >=3.10,<3.13; 0.119.0 Requires-Python >=3.10,<3.13; 0.120.0 Requires-Python >=3.10,<3.13; 0.120.1 Requires-Python >=3.10,<3.13; 0.121.0 Requires-Python >=3.10,<3.13; 0.121.1 Requires-Python >=3.10,<3.13; 0.14.0 Requires-Python >=3.10,<=3.13; 0.14.0rc0 Requires-Python >=3.10,<3.12; 0.14.0rc1 Requires-Python >=3.10,<=3.13; 0.14.1 Requires-Python >=3.10,<=3.13; 0.14.3 Requires-Python >=3.10,<=3.13; 0.14.4 Requires-Python >=3.10,<=3.13; 0.16.0 Requires-Python >=3.10,<=3.13; 0.16.1 Requires-Python >=3.10,<=3.13; 0.16.2 Requires-Python >=3

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import crewai library
from crewai import Agent, Task, Crew